## **LIBRARY IMPORTS**

In [14]:
# Import Libraries
import cudf
import os
import pywt
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb  # For reading MIT-BIH data
import keras_tuner as kt
import seaborn as sns
import tensorflow as tf
from collections import Counter
from scipy.signal import find_peaks, resample

# Scikit-learn and Imbalanced-learn imports
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    precision_recall_curve,
    auc,
    f1_score,
    precision_score,
    recall_score,
    accuracy_score
)
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import Pipeline as ImbPipeline
from scipy.stats import entropy

# Model imports
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Dropout, Add, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from cuml.ensemble import RandomForestClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.utils.class_weight import compute_class_weight

# Additional setups
# Checking cUML
print(cudf.Series([1, 2, 3]))

# Setting TensorFlow flags
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Checking GPU
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"TensorFlow has detected {len(gpu_devices)} GPU(s):")
    for device in gpu_devices:
        print(f"- {device}")
else:
    print("TensorFlow did not detect any GPUs. Training will run on the CPU.")


0    1
1    2
2    3
dtype: int64
TensorFlow has detected 1 GPU(s):
- PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## **DATA PREPARATION**

### DATA PREPARATION FUNCTIONS
There are 3 types of functions:
1. Labels and windowed features of the RR intervals
2. Data preparation of MIT-BIH dataset as training and validation set
3. Data preparation of additional ECG data with a format of .bin as the final testing set

In [15]:
# 1. LABELS & ADDITIONAL FUNCTIONS
label_map = { 'N': 0, 'L': 0, 'R': 0, 'e': 0, 'j': 0,  # Normal Beats (N)
              'V': 1, 'E': 1,                          # Ventricular Ectopic (VEB)
              'S': 2, 'A': 2, 'a': 2, 'J': 2,          # Supraventricular Ectopic (SVEB)
              'F': 3                                  # Fusion Beat (F)
            }
# Helper function for resampling signals to a new length
def resample_data(X, original_len, new_len):
    """Resamples all windows in X from original_len to new_len."""
    # Ensure X is 3D (num_samples, time_steps, channels)
    if X.ndim != 3:
        raise ValueError("Input array X must be 3-dimensional.")
    
    # Remove the channel dimension for resampling
    X_reshaped = X.squeeze(axis=-1)
    
    # Resample each signal window
    resampled_X = np.apply_along_axis(
        lambda row: resample(row, new_len),
        axis=1,
        arr=X_reshaped
    )
    
    # Add the channel dimension back
    return np.expand_dims(resampled_X, axis=-1)

# Extract features of wavelet for 1D-CNN
def extract_wavelet_features(window, wavelet='db4', level=4):
    """
    Extracts statistical and entropy-based features from the wavelet coefficients of an ECG window.
    
    Args:
        window (np.ndarray): A 1D numpy array representing a single ECG beat window.
        wavelet (str): The type of wavelet to use.
        level (int): The level of wavelet decomposition.
        
    Returns:
        np.ndarray: A 1D numpy array containing the extracted features.
    """
    # Decompose the signal
    coeffs = pywt.wavedec(window, wavelet, level=level)
    
    features = []
    for c in coeffs:
        # Basic statistical features
        features.append(np.mean(c))
        features.append(np.std(c))
        features.append(np.var(c))
        
        # Energy of the coefficients
        features.append(np.sum(np.square(c)))
        
        # Shannon Entropy of the coefficients
        # We use the squared coeffs to represent energy distribution for entropy calculation
        # Adding a small epsilon to avoid log(0)
        features.append(entropy(np.square(c) + 1e-9))
        
    return np.array(features)

# Wavelet data from MIT-BIH
def prepare_wavelet_data(signals, annotations, window_size, fs=360, wavelet='db4', level=4):
    """
    Prepares data by extracting wavelet-based features from ECG signal windows
    centered around each annotated R-peak.

    Args:
        signals (list): List of raw ECG signal arrays.
        annotations (list): List of wfdb Annotation objects.
        window_size (int): The total number of samples in each window.
        fs (int): The sampling frequency of the signals.
        wavelet (str): The type of wavelet to use for feature extraction.
        level (int): The level of wavelet decomposition.

    Returns:
        A tuple (X, y) where:
        - X is a 2D numpy array of wavelet features (num_beats, num_features).
        - y is a 1D numpy array of corresponding integer labels.
    """
    samples_before = window_size // 3
    samples_after = window_size - samples_before

    all_feature_vectors = []
    all_labels = []

    for i, signal in enumerate(signals):
        ann = annotations[i]
        r_peaks = ann.sample
        symbols = ann.symbol

        if ann.fs!= fs:
            print(f"Warning: Record {ann.record_name} has fs={ann.fs}, but expected fs={fs}. Skipping.")
            continue

        for j, r_peak_loc in enumerate(r_peaks):
            symbol = symbols[j]
            if symbol in label_map:
                start = r_peak_loc - samples_before
                end = r_peak_loc + samples_after

                if start >= 0 and end < len(signal):
                    window = signal[start:end]
                    label = label_map[symbol]
                    
                    # Extract features from the window
                    features = extract_wavelet_features(window, wavelet=wavelet, level=level)
                    
                    all_feature_vectors.append(features)
                    all_labels.append(label)

    X = np.array(all_feature_vectors)
    y = np.array(all_labels)

    print(f"Successfully created feature matrix with shape: {X.shape}")
    return X, y

# Wavelet data from raw ECG data with format of .bin
def prepare_wavelet_data_from_bin(signal, r_peaks, window_size, record_label, wavelet='db4', level=4):
    """
    Prepares data by extracting wavelet features from windows around R-peaks from a.bin file.

    Args:
        signal (numpy.ndarray): The raw ECG signal array.
        r_peaks (numpy.ndarray): The indices of the detected R-peaks.
        window_size (int): The total number of samples in each window.
        record_label (int): The integer label to apply to all windows.
        wavelet (str): The type of wavelet to use for feature extraction.
        level (int): The level of wavelet decomposition.

    Returns:
        A tuple (X, y) where:
        - X is a 2D numpy array of wavelet features.
        - y is a 1D numpy array of corresponding integer labels.
    """
    print(f"\n--- Step 2: Extracting wavelet features from signal windows ---")
    samples_before = window_size // 3
    samples_after = window_size - samples_before

    all_feature_vectors = []
    all_labels = []

    for r_peak_loc in r_peaks:
        start = r_peak_loc - samples_before
        end = r_peak_loc + samples_after

        if start >= 0 and end < len(signal):
            window = signal[start:end]
            
            # Extract features from the window
            features = extract_wavelet_features(window, wavelet=wavelet, level=level)
            
            all_feature_vectors.append(features)
            all_labels.append(record_label)

    X = np.array(all_feature_vectors)
    y = np.array(all_labels)

    print(f"Successfully created {len(X)} feature vectors.")
    print(f"Final feature matrix shape (X): {X.shape}")
    print(f"Final labels shape (y): {y.shape}")

    return X, y

In [16]:
# 2. DATA LOADING UTILITY OF MIT-BIH DATA
# This function efficiently loads the specified records and their annotations.
def load_mitbih_records(db_path, record_names):
    """
    Loads raw ECG signals and annotations for specified records.

    Args:
        db_path (str): The path to the database directory (e.g., 'mit-bih-arrhythmia-database-1.0.0').
        record_names (list): A list of record names as strings (e.g., ['100', '101']).

    Returns:
        A tuple containing two lists: (signals, annotations).
        - signals: A list of raw ECG signal arrays for channel 0.
        - annotations: A list of wfdb Annotation objects.
    """
    all_signals = []
    all_annotations = []
    print(f"Loading records: {', '.join(record_names)}...")
    for rec_name in record_names:
        record_path = f'{db_path}/{rec_name}'
        try:
            # Read the signal from the first channel (.dat file)
            signal = wfdb.rdrecord(record_path, channels=[0]).p_signal.flatten()
            # Read the corresponding annotations (.atr file)
            annotation = wfdb.rdann(record_path, 'atr')

            all_signals.append(signal)
            all_annotations.append(annotation)
        except Exception as e:
            print(f"Error processing record {rec_name}: {e}")
    print("Loading complete.")
    return all_signals, all_annotations

# A function to extract raw signal windows (beats) for model training.
def prepare_raw_data(signals, annotations, window_size, fs=360):
    """
    Prepares data by extracting fixed-size raw ECG signal windows
    centered around each annotated R-peak.

    Args:
        signals (list): List of raw ECG signal arrays.
        annotations (list): List of wfdb Annotation objects.
        window_size (int): The total number of samples in each window (e.g., 288).
        fs (int): The sampling frequency of the signals (default is 360 for MIT-BIH).

    Returns:
        A tuple (X, y) where:
        - X is a numpy array of ECG signal windows, ready for a CNN.
        - y is a numpy array of corresponding integer labels.
    """
    # A common practice is to place the R-peak off-center (e.g., at 1/3)
    # for better feature extraction by the model.
    samples_before = window_size // 3
    samples_after = window_size - samples_before

    all_windows = []
    all_labels = []

    # Process each signal and its corresponding annotations
    for i, signal in enumerate(signals):
        ann = annotations[i]
        r_peaks = ann.sample
        symbols = ann.symbol

        # Verify the record's sampling frequency
        if ann.fs != fs:
            print(f"Warning: Record {ann.record_name} has fs={ann.fs}, but expected fs={fs}. Skipping.")
            continue

        # Extract a window for each valid R-peak annotation
        for j, r_peak_loc in enumerate(r_peaks):
            symbol = symbols[j]
            # Classify only the beats defined in our label_map
            if symbol in label_map:
                start = r_peak_loc - samples_before
                end = r_peak_loc + samples_after

                # Ensure the window is fully within the signal's bounds
                if start >= 0 and end < len(signal):
                    window = signal[start:end]
                    label = label_map[symbol]
                    all_windows.append(window)
                    all_labels.append(label)

    # Convert lists to numpy arrays
    X = np.array(all_windows)
    y = np.array(all_labels)

    # Add a "channel" dimension for compatibility with deep learning models (e.g., CNNs)
    # The shape becomes (number_of_beats, window_size, 1)
    X = np.expand_dims(X, axis=-1)

    return X, y

In [17]:
# 3. DATA LOADING FOR .bin FILES
# A function to load ECG data
def load_ecg_from_bin(file_path, dtype=np.int16):
    """
    Loading raw ECG signals from binary files.

    Args:
        file_path (str): Path to the .bin file.
        dtype (numpy.dtype): Data type of the signal in the .bin file.

    Return:
        numpy.ndarray: ECG signals as a numpy array.
    """
    try:
        signal = np.fromfile(file_path, dtype=dtype)
        print(f"Completed reading {len(signal)} samples from {file_path}")
        return signal
    except IOError as e:
        print(f"An error has occurred while reading: {e}")
        return None

# A function to detect R-peaks for labelling
def detect_r_peaks(signal, fs):
    """
    Detecting R-peaks from the ECG signal. This serves as our substitute
    for reading an annotation file.

    Args:
        signal (numpy.ndarray): Raw ECG signal in a numpy array.
        fs (int): Sampling frequency of the ECG signal.

    Return:
        numpy.ndarray: An array of indices of the detected R-peaks.
    """
    print("\n--- Step 1: Detecting R-Peaks to locate heartbeats ---")
    height_threshold = np.max(signal) * 0.6
    distance_threshold = fs * 0.4
    r_peaks, _ = find_peaks(signal, height=height_threshold, distance=distance_threshold)
    print(f"Detected {len(r_peaks)} R-peaks.")
    return r_peaks

# A function to prepare raw ECG data with its R-peaks
def prepare_raw_data_from_bin(signal, r_peaks, window_size, record_label):
    """
    Prepares data by extracting fixed-size raw ECG signal windows
    centered around each detected R-peak from a .bin file.

    Args:
        signal (numpy.ndarray): The raw ECG signal array.
        r_peaks (numpy.ndarray): The indices of the detected R-peaks.
        window_size (int): The total number of samples in each window.
        record_label (int): The integer label to apply to all windows from this record.

    Returns:
        A tuple (X, y) where:
        - X is a numpy array of ECG signal windows, ready for a CNN.
        - y is a numpy array of corresponding integer labels.
    """
    print(f"\n--- Step 2: Extracting raw signal windows ---")
    samples_before = window_size // 3
    samples_after = window_size - samples_before

    all_windows = []
    all_labels = []

    for r_peak_loc in r_peaks:
        start = r_peak_loc - samples_before
        end = r_peak_loc + samples_after

        # Ensure the window is fully within the signal's bounds
        if start >= 0 and end < len(signal):
            window = signal[start:end]
            all_windows.append(window)
            all_labels.append(record_label)

    # Convert lists to numpy arrays
    X = np.array(all_windows)
    y = np.array(all_labels)

    # Add a "channel" dimension for compatibility with deep learning models (e.g., CNNs)
    # The shape becomes (number_of_beats, window_size, 1)
    X = np.expand_dims(X, axis=-1)

    print(f"Successfully created {len(X)} windows of size {window_size}.")
    print(f"Final data shape (X): {X.shape}")
    print(f"Final labels shape (y): {y.shape}")

    return X, y

# A function to visualize the data
def plot_beat_windows(X, y, n=5):
    """
    Plots a few sample beat windows.

    Args:
        X (numpy.ndarray): The data windows.
        y (numpy.ndarray): The labels.
        n (int): Number of samples to plot.
    """
    plt.figure(figsize=(15, 10))
    for i in range(n):
        plt.subplot(n, 1, i + 1)
        plt.plot(X[i].flatten()) # flatten to remove the channel dimension for plotting
        plt.title(f"Sample Beat Window {i+1} - Label: {y[i]}")
        plt.ylabel("Amplitude")
    plt.xlabel("Sample Number within Window")
    plt.tight_layout()
    plt.show()

### DATA PREPARATION EXECUTION
1. Reading all ECG datasets
2. Divide all datasets into training dataset and testing dataset
3. Standard scaling and combining datasets
4. Splitting training dataset into training split and validation split then applying SMOTE algorithm into the training split
5. Preparing all the datasets for each machine learning model

In [18]:
    # --- Configuration Section ---
    DB_PATH_MIT = '../data/raw/MIT-BIH/mit-bih-arrhythmia-database-1.0.0/mit-bih-arrhythmia-database-1.0.0/'
    FS_MIT = 360
    WINDOW_SIZE_MIT = 288  # 800ms window -> 0.8s * 360Hz

    # Wavelet Feature Configuration
    WAVELET_TYPE = 'db4'
    WAVELET_LEVEL = 4

    # Split MIT-BIH records into training and testing sets to prevent patient data leakage
    RECORDS_TRAIN = ['101', '106', '108', '109', '112', '114', '115', '116', '118', '119',
                     '122', '124', '201', '203', '205', '207', '208', '209', '215', '220',
                     '223', '230'] # DS1
    RECORDS_TEST = ['100', '103', '105', '111', '113', '117', '121', '123', '200', '202',
                    '210', '212', '213', '214', '219', '221', '222', '228', '231', '232',
                    '233', '234'] # DS2

    FS_CUSTOM = 500
    WINDOW_SIZE_CUSTOM = 400  # 800ms window -> 0.8s * 500Hz
    custom_file_paths = {
        'Arrhythmia': '../data/raw/Arrhythmia/ECG_WAVE.bin',
        'Normal': '../data/raw/Normal/ecg_normal.bin' 
    }
    custom_file_labels = {'Arrhythmia': 2, 'Normal': 0} # SVEB and Normal

    # ===================================================================
    # Data Preparation Pipeline (Using Wavelet Features)
    # ===================================================================

    # --- [Step 1/4] Extract Wavelet Features ---
    print("--- [Step 1/4] Extracting Wavelet Features from All Datasets ---")
    
    # Load MIT-BIH training set (DS1) and extract features
    print("\nProcessing MIT-BIH Training set (DS1)...")
    train_signals_mit, train_anns_mit = load_mitbih_records(DB_PATH_MIT, RECORDS_TRAIN)
    X_train_mit, y_train_mit = prepare_wavelet_data(
        train_signals_mit, train_anns_mit, WINDOW_SIZE_MIT, fs=FS_MIT, wavelet=WAVELET_TYPE, level=WAVELET_LEVEL
    )

    # Load MIT-BIH testing set (DS2) and extract features
    print("\nProcessing MIT-BIH Testing set (DS2)...")
    test_signals_mit, test_anns_mit = load_mitbih_records(DB_PATH_MIT, RECORDS_TEST)
    X_test_mitbih, y_test_mitbih = prepare_wavelet_data(
        test_signals_mit, test_anns_mit, WINDOW_SIZE_MIT, fs=FS_MIT, wavelet=WAVELET_TYPE, level=WAVELET_LEVEL
    )
    
    # Load custom data, extract features, and designate it for testing
    print("\nProcessing custom .bin files for the TESTING set...")
    X_test_custom_list, y_test_custom_list = [], []

    for name, path in custom_file_paths.items():
        label = custom_file_labels[name]
        signal = load_ecg_from_bin(path)
        if signal is not None:
            r_peaks = detect_r_peaks(signal, fs=FS_CUSTOM)
            X_single, y_single = prepare_wavelet_data_from_bin(
                signal, r_peaks, WINDOW_SIZE_CUSTOM, label, wavelet=WAVELET_TYPE, level=WAVELET_LEVEL
            )
            X_test_custom_list.append(X_single)
            y_test_custom_list.append(y_single)
            
    # Combine all custom data into a single feature matrix
    X_test_custom = np.vstack(X_test_custom_list)
    y_test_custom = np.concatenate(y_test_custom_list)

    # --- [Step 2/4] Scaling & Combining Testing Data ---
    print("\n--- [Step 2/4] Scaling and Finalizing Data ---")
    
    # The training data is ONLY from MIT-BIH DS1.
    X_train = X_train_mit
    y_train = y_train_mit
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    print("Scaler trained on training data.")
    
    X_test_mitbih_scaled = scaler.transform(X_test_mitbih)
    X_test_custom_scaled = scaler.transform(X_test_custom)
    print("Scaler applied to all testing data.")
    
    X_test_final = np.concatenate((X_test_mitbih_scaled, X_test_custom_scaled), axis=0)
    y_test_final = np.concatenate((y_test_mitbih, y_test_custom), axis=0)
    print(f"Final training data created: X_train_scaled={X_train_scaled.shape}, y_train={y_train.shape}")
    print(f"Final testing data combined: X_test_final={X_test_final.shape}, y_test_final={y_test_final.shape}")

    # --- [Step 3/4] Training Set Splitting & Hybrid Sampling (SMOTEENN) ---
    print("\n--- [Step 3/4] Finalizing Training Data (Split & Hybrid Sample) ---")
    # Determine number of classes from all available labels
    output_dim = len(np.unique(np.concatenate((y_train, y_test_final))))
    
    print("Creating validation set from training data (80/20)...")
    X_train_fold, X_val, y_train_fold, y_val = train_test_split(
        X_train_scaled, y_train, test_size=0.2, random_state=42, stratify=y_train
    )

    print("Applying HYBRID SAMPLING (SMOTEENN) only to the training fold...")
    print("Training class distribution before sampling:", Counter(y_train_fold))
    
    sampler = SMOTEENN(random_state=42)
    X_train_resampled, y_train_resampled = sampler.fit_resample(X_train_fold, y_train_fold)
    
    print("Training class distribution after SMOTEENN sampling:", Counter(y_train_resampled))

    # --- [Step 4/4] Final Data Preparation for Models ---
    print("\n--- [Step 4/4] Preparing Final Datasets for Models ---")

    # One-hot encoding labels for Keras/TensorFlow models
    y_train_encoded = to_categorical(y_train_resampled, num_classes=output_dim)
    y_val_encoded = to_categorical(y_val, num_classes=output_dim)
    y_test_final_encoded = to_categorical(y_test_final, num_classes=output_dim)

    # 🧠 Data for MLP (requires 2D input)
    X_train_mlp, y_train_mlp = X_train_resampled, y_train_encoded
    X_val_mlp, y_val_mlp = X_val, y_val_encoded
    X_test_mlp, y_test_mlp = X_test_final, y_test_final_encoded

    # ⚡ Data for 1D-CNN (requires 3D input: samples, feature_vector_length, channels)
    X_train_cnn = X_train_mlp.reshape((X_train_mlp.shape[0], X_train_mlp.shape[1], 1))
    X_val_cnn = X_val_mlp.reshape((X_val_mlp.shape[0], X_val_mlp.shape[1], 1))
    X_test_cnn = X_test_mlp.reshape((X_test_mlp.shape[0], X_test_mlp.shape[1], 1))
    y_train_cnn, y_val_cnn, y_test_cnn = y_train_mlp, y_val_mlp, y_test_mlp

    # 🌲 Data for RandomForest (requires 2D input and integer labels)
    X_train_rf, y_train_rf = X_train_resampled, y_train_resampled
    X_val_rf, y_val_rf = X_val, y_val
    X_test_rf, y_test_rf = X_test_final, y_test_final

    # --- FINAL RESULTS ---
    print("\n" + "="*60)
    print("✅ WAVELET DATA PREPARATION COMPLETE ✅")
    print("The following variables are ready for training and evaluation:")
    print("="*60)
    
    print("\n--- For MLP ---")
    print(f"  Training:   X_train_mlp: {X_train_mlp.shape}, y_train_mlp: {y_train_mlp.shape}")
    print(f"  Validation: X_val_mlp:   {X_val_mlp.shape}, y_val_mlp:   {y_val_mlp.shape}")
    print(f"  Testing:    X_test_mlp:  {X_test_mlp.shape}, y_test_mlp:  {y_test_mlp.shape}")

    print("\n--- For 1D-CNN ---")
    print(f"  Training:   X_train_cnn: {X_train_cnn.shape}, y_train_cnn: {y_train_cnn.shape}")
    print(f"  Validation: X_val_cnn:   {X_val_cnn.shape}, y_val_cnn:   {y_val_cnn.shape}")
    print(f"  Testing:    X_test_cnn:  {X_test_cnn.shape}, y_test_cnn:  {y_test_cnn.shape}")

    print("\n--- For RandomForest ---")
    print(f"  Training:   X_train_rf: {X_train_rf.shape}, y_train_rf: {y_train_rf.shape}")
    print(f"  Validation: X_val_rf:   {X_val_rf.shape}, y_val_rf:   {y_val_rf.shape}")
    print(f"  Testing:    X_test_rf:  {X_test_rf.shape}, y_test_rf:  {y_test_rf.shape}")

--- [Step 1/4] Extracting Wavelet Features from All Datasets ---

Processing MIT-BIH Training set (DS1)...
Loading records: 101, 106, 108, 109, 112, 114, 115, 116, 118, 119, 122, 124, 201, 203, 205, 207, 208, 209, 215, 220, 223, 230...
Loading complete.
Successfully created feature matrix with shape: (50993, 25)

Processing MIT-BIH Testing set (DS2)...
Loading records: 100, 103, 105, 111, 113, 117, 121, 123, 200, 202, 210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234...
Loading complete.
Successfully created feature matrix with shape: (49683, 25)

Processing custom .bin files for the TESTING set...
Completed reading 2380000 samples from ../data/raw/Arrhythmia/ECG_WAVE.bin

--- Step 1: Detecting R-Peaks to locate heartbeats ---
Detected 137 R-peaks.

--- Step 2: Extracting wavelet features from signal windows ---
Successfully created 136 feature vectors.
Final feature matrix shape (X): (136, 25)
Final labels shape (y): (136,)
Completed reading 2135000 samples from ../data/raw/N

## **MACHINE LEARNING MODEL TRAINING**

### MACHINE LEARNING MODEL FUNCTIONS
There are 3 models that will be trained:
1. MLP Model (TA242501010)
2. 1D-CNN
3. RandomForest

There is also an additional function in order to do automatic hyperparameter tuning, but the function is only made for the MLP model.

In [19]:
# Function to build the MLP model for automatic hyperparameter tuning
def build_model(hp):
    """Function that builds a Keras model and defines the hyperparameters to be tuned."""
    model = Sequential()

    # Tune the number of units in the first hidden layer
    hp_units_1 = hp.Int('units_1', min_value=32, max_value=256, step=32)
    model.add(Dense(units=hp_units_1, activation='relu', input_shape=(X_train.shape[1],)))

    # Tune the dropout rate
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    model.add(Dropout(rate=hp_dropout_1))

    # Tune the number of units in the second hidden layer
    hp_units_2 = hp.Int('units_2', min_value=32, max_value=256, step=32)
    model.add(Dense(units=hp_units_2, activation='relu'))

    # Tune the dropout rate
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    model.add(Dropout(hp_dropout_2))
    model.add(Dense(output_dim, activation='softmax'))

    # Tune the learning rate for the Adam optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc_roc'),
            tf.keras.metrics.AUC(name='auc_pr', curve='PR'),
            tf.keras.metrics.F1Score(average='weighted', name='f1_score')
        ]
    )
    return model

# Function to create the MLP model for cross-validation
def create_mlp_model(input_dim, output_dim):
    """Creates and compiles a Keras MLP model."""
    model = Sequential([
        # Hyperparameters tuning
        Dense(512, input_dim=input_dim, activation='relu'),
        Dropout(0.1),
        Dense(512, activation='relu'),
        Dropout(0.4),
        Dense(output_dim, activation='softmax') # Softmax for multi-class classification
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy', # Suitable for one-hot labels
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.F1Score(average='weighted', name='f1_score'),
            tf.keras.metrics.SpecificityAtSensitivity(0.9, name='specificity')
        ]
    )
    return model
# Function to create the 1D-CNN model
def create_cnn_model(input_shape, output_dim):
    """Creates and compiles a Keras 1D-CNN model."""
    # Input shape for CNN must be 3D: (samples, steps, features)
    # Example: (10000, 187, 1)

    model = Sequential([
        Conv1D(filters=256, kernel_size=6, activation='relu', # Reduced filters
               input_shape=input_shape),
        Dropout(0.1),
        MaxPooling1D(pool_size=2),

        Conv1D(filters=256, kernel_size=3, activation='relu'), # Reduced filters
        Dropout(0.2),
        MaxPooling1D(pool_size=2),

        Flatten(), # Now flattens a much smaller tensor

        Dense(256, activation='relu'), # Reduced dense units
        Dropout(0.4),

        Dense(output_dim, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.F1Score(average='weighted', name='f1_score'),
            tf.keras.metrics.SpecificityAtSensitivity(0.9, name='specificity')
        ]
    )
    return model

def create_cnn_model_optimized(input_shape, output_dim):
    """
    Creates and compiles an optimized 1D-CNN model using modern architectural patterns
    like Batch Normalization, Global Pooling, and Residual Connections.

    Args:
        input_shape (tuple): The shape of the input data (e.g., (num_features, 1)).
        output_dim (int): The number of output classes.

    Returns:
        A compiled Keras Model.
    """
    
    # --- Define a reusable Residual Block ---
    def residual_block(x, filters, kernel_size):
        """A residual block with two convolutional layers."""
        # Main path
        y = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        
        y = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(y)
        y = BatchNormalization()(y)
        
        # Shortcut connection: Add the input 'x' to the output of the block
        # The input and output must have the same dimensions for addition.
        # If not, a 1x1 convolution is used to match them.
        if x.shape[-1] != filters:
            shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(x)
        else:
            shortcut = x
            
        res_output = Add()([shortcut, y])
        res_output = Activation('relu')(res_output)
        return res_output

    # --- Build the Model ---
    inputs = Input(shape=input_shape)

    # Initial Convolutional Layer
    # This layer processes the raw input features
    x = Conv1D(filters=128, kernel_size=7, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # --- Stack of Residual Blocks ---
    # Each block learns progressively more complex features
    x = residual_block(x, filters=128, kernel_size=5)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.2)(x) # Dropout between blocks for regularization

    x = residual_block(x, filters=256, kernel_size=3)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)

    # --- Transition to Classifier Head ---
    # Recommendation 2: Use Global Pooling instead of Flatten
    # This drastically reduces parameters and helps prevent overfitting.
    x = GlobalAveragePooling1D()(x)

    # --- Dense Classifier Head ---
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # --- Output Layer ---
    outputs = Dense(output_dim, activation='softmax')(x)

    # Create and compile the final model
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            # Note: F1Score might require a different setup in some TF versions.
            # If it causes issues, consider a custom callback to calculate it.
            tf.keras.metrics.F1Score(average='weighted', name='f1_score'),
        ]
    )
    return model

# Function to create the RandomForest model
def create_rf_model():
    """Creates an instance of the GPU-accelerated RandomForestClassifier model using cuML."""
    # Hyperparameters are similar to scikit-learn's
    model = RandomForestClassifier(
        n_estimators=180,  # Number of trees in the forest
        random_state=42,
        max_depth=30       # Note: The 'n_jobs' parameter is not needed as it runs on the GPU
    )
    return model

### MACHINE LEARNING MODEL TRAINING EXECUTION
1. Targeted metrics: Precision, Recall, F1-Score, and Specificity
2. There are 3 executions: multiple models training, MLP model specific training, and MLP model automatic hyperparameter tuning

In [ ]:
# Training Multiple Models
input_shape_cnn = (X_train_cnn.shape[1], X_train_cnn.shape[2])
input_dim = X_train_mlp.shape[1]
# output_dim already defined from the DATA PREPARATION section

models = {
    # "1D-CNN": create_cnn_model(input_shape_cnn, output_dim),
    "1D-CNN": create_cnn_model_optimized(input_shape_cnn, output_dim),
    "RandomForest": create_rf_model(),
    "MLP": create_mlp_model(input_dim, output_dim)
}

# Dictionary to store the final results
results = {}

# --- TRAINING AND EVALUATING EACH MODEL ---

for name, model in models.items():
    print(f"\n{'='*20} TRAINING MODEL: {name} {'='*20}")

    # 🧠 Training
    if name == "1D-CNN":
        model.fit(
            X_train_cnn, y_train_cnn,
            epochs=10,
            batch_size=32,
            verbose=1,
            validation_data=(X_val_cnn, y_val_cnn) # Using the existing validation set
        )
    elif name == "MLP":
        model.fit(
            X_train_mlp, y_train_mlp,
            epochs=10,
            batch_size=32,
            verbose=1,
            validation_data=(X_val_mlp, y_val_mlp) # Using the existing validation set
        )
    else: # 📊 RandomForest
        model.fit(X_train_rf, y_train_rf)

    # ⚡ Prediction on the Test Set
    print(f"Evaluating model {name}...")
    if name == "MLP":
        y_pred_raw = model.predict(X_test_mlp)
        y_pred = np.argmax(y_pred_raw, axis=1)
    elif name == "1D-CNN":
        y_pred_raw = model.predict(X_test_cnn)
        y_pred = np.argmax(y_pred_raw, axis=1)
    else: # RandomForest
        y_pred = model.predict(X_test_rf)

    # Save prediction results for final evaluation
    results[name] = {'y_pred': y_pred}

# --- PRINT ALL RESULTS SIMULTANEOUSLY ---

class_names = ['Normal (N)', 'Ventricular (V)', 'Supraventricular (S)', 'Fusion (F)', 'Unknown (Q)']

print(f"\n{'='*25} FINAL EVALUATION RESULTS {'='*25}")

for name, result_data in results.items():
    y_pred_test = result_data['y_pred']

    print(f"\n\n--- REPORT FOR MODEL: {name} ---")
    # Standard Classification Report (using y_test_final, the original integer labels)
    print("\nClassification Report on the Test Set:")
    print(classification_report(y_test_final, y_pred_test, target_names=class_names))

    # Additional Metrics Report
    print("Additional Metrics Report:")
    cm = confusion_matrix(y_test_final, y_pred_test)
    
    for i in range(len(class_names)):
        tn = cm.sum() - (cm[i,:].sum() + cm[:,i].sum() - cm[i,i])
        tp = cm[i,i]
        fp = cm[:,i].sum() - cm[i,i]
        fn = cm[i,:].sum() - cm[i,i]

        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

        print(f"  Class: {class_names[i]}")
        print(f"    - Sensitivity (Recall): {sensitivity:.4f}")
        print(f"    - Specificity         : {specificity:.4f}")
        print(f"    - False Positive Rate : {fpr:.4f}")


==================== TRAINING MODEL: 1D-CNN ====================
Epoch 1/10


/mnt/c/Users/OSOTNAS/Documents/Kean/Others/KP_Xirka-Darma-Persada/machine-learning/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1755135277.454056    1902 service.cc:152] XLA service 0x72a3e00021f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755135277.454134    1902 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2025-08-14 08:34:37.589767: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755135278.143348    1902 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-1

  22/4471 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - accuracy: 0.2758 - f1_score: 0.2492 - loss: 2.1856 - precision: 0.2998 - recall: 0.2125

I0000 00:00:1755135284.770392    1902 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4471/4471 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7691 - f1_score: 0.7686 - loss: 0.6047 - precision: 0.8097 - recall: 0.7195

2025-08-14 08:35:15.752031: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_298', 4 bytes spill stores, 4 bytes spill loads



4471/4471 ━━━━━━━━━━━━━━━━━━━━ 44s 8ms/step - accuracy: 0.8618 - f1_score: 0.8616 - loss: 0.3681 - precision: 0.8852 - recall: 0.8388 - val_accuracy: 0.8529 - val_f1_score: 0.8935 - val_loss: 0.3860 - val_precision: 0.8592 - val_recall: 0.8456
Epoch 2/10
4471/4471 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - accuracy: 0.9412 - f1_score: 0.9412 - loss: 0.1611 - precision: 0.9448 - recall: 0.9376 - val_accuracy: 0.8242 - val_f1_score: 0.8711 - val_loss: 0.4849 - val_precision: 0.8271 - val_recall: 0.8194
Epoch 3/10
4471/4471 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - accuracy: 0.9579 - f1_score: 0.9579 - loss: 0.1168 - precision: 0.9599 - recall: 0.9561 - val_accuracy: 0.8818 - val_f1_score: 0.9060 - val_loss: 0.3530 - val_precision: 0.8843 - val_recall: 0.8784
Epoch 4/10
4471/4471 ━━━━━━━━━━━━━━━━━━━━ 29s 7ms/step - accuracy: 0.9673 - f1_score: 0.9672 - loss: 0.0915 - precision: 0.9684 - recall: 0.9662 - val_accuracy: 0.8690 - val_f1_score: 0.9033 - val_loss: 0.3939 - val_precision: 0.8713 - val_recall

2025-08-14 08:39:48.674708: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_132_0', 44 bytes spill stores, 44 bytes spill loads

2025-08-14 08:39:48.775540: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_139', 4 bytes spill stores, 4 bytes spill loads

2025-08-14 08:39:48.937550: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_132', 484 bytes spill stores, 484 bytes spill loads

2025-08-14 08:39:49.175732: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_139', 4 bytes spill stores, 4 bytes spill loads

2025-08-14 08:39:52.212998: I external/local

4471/4471 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.8530 - f1_score: 0.8526 - loss: 0.4007 - precision: 0.8863 - recall: 0.8129 - specificity: 0.9256 - val_accuracy: 0.7687 - val_f1_score: 0.8320 - val_loss: 0.5274 - val_precision: 0.7809 - val_recall: 0.7498 - val_specificity: 0.8679
Epoch 2/10
4471/4471 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.9174 - f1_score: 0.9173 - loss: 0.2314 - precision: 0.9262 - recall: 0.9086 - specificity: 0.9781 - val_accuracy: 0.8218 - val_f1_score: 0.8684 - val_loss: 0.4527 - val_precision: 0.8301 - val_recall: 0.8129 - val_specificity: 0.9026
Epoch 3/10
4471/4471 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.9361 - f1_score: 0.9360 - loss: 0.1796 - precision: 0.9420 - recall: 0.9303 - specificity: 0.9875 - val_accuracy: 0.8464 - val_f1_score: 0.8843 - val_loss: 0.4082 - val_precision: 0.8530 - val_recall: 0.8383 - val_specificity: 0.9253
Epoch 4/10
3873/4471 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9438 - f1_score: 0.9437 - loss: 0.

In [ ]:
# Training MLP Model - TA242501010
# Initialize AI model
model = create_mlp_model(input_dim, output_dim)

# Prepare EarlyStopping callback for F1 score validation
early_stopping_1 = EarlyStopping(monitor='val_f1_score', patience=5, restore_best_weights=True)

# Prepare class weights
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

# Start AI model training
history = model.fit(
    X_train_resampled,
    y_train_resampled_encoded,
    epochs=20,
    batch_size=16,
    validation_data=(X_val_fold, y_val_fold_encoded),
    verbose=1,
    # class_weight=class_weight_dict,
    validation_split=0.2
    # callbacks=[early_stopping_1]
)

# Evaluate on the untouched set
print("Evaluating on the untouched test set...")
X_test_scaled = scaler.transform(X_test)
y_pred_test_raw = model.predict(X_test_scaled)

# Convert predictions back to labels if they are one-hot encoded
if hasattr(y_pred_test_raw, 'shape') and len(y_pred_test_raw.shape) > 1:
      y_pred_test = np.argmax(y_pred_test_raw, axis=1)
else:
      y_pred_test = y_pred_test_raw

class_names = ['Normal (N)', 'Ventricular (V)', 'Supraventricular (S)', 'Fusion (F)', 'Unknown (Q)']

print("\nClassification Report on the Test Set:")
print(classification_report(y_test, y_pred_test, target_names=class_names))

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred_test)

# Calculate metrics for each class (one-vs-rest)
print("\nAdditional Metrics Report:")
print("="*55)
for i in range(len(class_names)):
    tn = cm.sum() - (cm[i,:].sum() + cm[:,i].sum() - cm[i,i])
    tp = cm[i,i]
    fp = cm[:,i].sum() - cm[i,i]
    fn = cm[i,:].sum() - cm[i,i]

    # Sensitivity (Recall)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    # Specificity
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    # False Positive Rate
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

    print(f"Class: {class_names[i]}")
    print(f"  Sensitivity (Recall): {sensitivity:.4f}")
    print(f"  Specificity         : {specificity:.4f}")
    print(f"  False Positive Rate : {fpr:.4f}")
    print("-"*25)

In [ ]:
# Automatic Hyperparameter Tuning
print("\n--- Starting Automatic Hyperparameter Tuning with KerasTuner ---")

# Calculate class_weight only once
# class_weights = compute_class_weight(
#     'balanced',
#     classes=np.unique(y_train),
#     y=y_train
# )
# class_weight_dict = dict(enumerate(class_weights))

# Defining tuner objectives
multi_objectives = [
    kt.Objective("val_f1_score", direction="max")
    #kt.Objective("val_specificity", direction="max")
    #kt.Objective("val_precision", direction="max")
    #kt.Objective("val_recall", direction="max")
    #kt.Objective("val_auc_roc", direction="max")
    #kt.Objective("val_auc_pr", direction="max")
]
# Initialize Tuner with RandomSearch
tuner = kt.RandomSearch(
    build_model,
    objective=multi_objectives, # Target: maximize validation F1 score
    max_trials=20,              # Total number of hyperparameter combinations to be tried
    executions_per_trial=1,     # Number of models trained per combination (for stability)
    directory='keras_tuner_dir',
    project_name='ecg_classification_0834' # Can change the name to find the latest parameters with the latest code
)

# Prepare EarlyStopping callback for F1 score validation
early_stopping = EarlyStopping(monitor='val_f1_score', patience=5, restore_best_weights=True)

# Run the search
print("\nStarting the search for the best hyperparameters...")
tuner.search(
    X_train_resampled,              # Training data that has been processed with SMOTE
    y_train_resampled_encoded,
    epochs=100,
    validation_data=(X_val_fold, y_val_fold_encoded), # 1. Use validation_data
    # class_weight=class_weight_dict,
    callbacks=[early_stopping],
    verbose=1
)

# Get the best hyperparameters and the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
--- Search Complete ---
Best hyperparameters found:
- Units 1: {best_hps.get('units_1')}
- Dropout 1: {best_hps.get('dropout_1'):.2f}
- Units 2: {best_hps.get('units_2')}
- Dropout 2: {best_hps.get('dropout_2'):.2f}
- Learning Rate: {best_hps.get('learning_rate')}
""")

# --- Final Evaluation on the Test Set ---
print("\n--- Evaluating the Best Model on the Test Set ---")
X_test_scaled = scaler.transform(X_test) # Use the same scaler from training
y_pred_test_raw = best_model.predict(X_test_scaled)
y_pred_test = np.argmax(y_pred_test_raw, axis=1)
class_names = ['Normal (N)', 'Ventricular (V)', 'Supraventricular (S)', 'Fusion (F)', 'Unknown (Q)']

# --- 1. Classification Report ----
print("\nClassification Report on the Test Set:")
print(classification_report(y_test, y_pred_test, target_names=class_names))

# --- 2. Confusion Matrix ---
print("\n--- Confusion Matrix ---")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

# --- 3. Specific Metric Calculation per Class ---
print("\n--- Detailed Performance Metrics per Class ---")
metrics_data = []

for i, class_name in enumerate(class_names):
    # Specificity calculation
    tn = cm.sum() - (cm[i,:].sum() + cm[:,i].sum() - cm[i,i])
    fp = cm[:,i].sum() - cm[i,i]
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    metrics_data.append({
        "Class": class_name,
        "Precision": precision_score(y_test, y_pred_test, average=None)[i],
        "Sensitivity (Recall)": recall_score(y_test, y_pred_test, average=None)[i],
        "F1-Score": f1_score(y_test, y_pred_test, average=None)[i],
        "Specificity": specificity
    })

metrics_df = pd.DataFrame(metrics_data)
print(metrics_df.to_string())


# --- 4. Calculation of AUC-ROC and AUC-PR (One-vs-Rest) ---
y_test_encoded = to_categorical(y_test, num_classes=output_dim)

# Add this line to define y_pred_proba
y_pred_proba = best_model.predict(X_test_scaled)

# AUC-ROC
auc_roc_ovr = roc_auc_score(y_test_encoded, y_pred_proba, multi_class='ovr', average='weighted')
print(f"\nAUC-ROC (One-vs-Rest, Weighted): {auc_roc_ovr:.4f}")

# AUC-PR
# Calculate for each class and average
precision_curves = dict()
recall_curves = dict()
auc_pr_scores = []
for i in range(output_dim):
    precision_curves[i], recall_curves[i], _ = precision_recall_curve(y_test_encoded[:, i], y_pred_proba[:, i])
    auc_pr_scores.append(auc(recall_curves[i], precision_curves[i]))

# Weighted average for AUC-PR
support = np.bincount(y_test)
avg_auc_pr = np.average(auc_pr_scores, weights=support)
print(f"AUC-PR (One-vs-Rest, Weighted): {avg_auc_pr:.4f}")

# Plotting PR Curves for each class
plt.figure(figsize=(10, 8))
for i, class_name in enumerate(class_names):
    plt.plot(recall_curves[i], precision_curves[i], lw=2, label=f'{class_name} (AUC-PR = {auc_pr_scores[i]:.2f})')

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve per Class")
plt.legend(loc="best")
plt.grid(True)
plt.show()